# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities = pd.read_csv('cities.csv', encoding = 'utf-8')
cities.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key = g_key)
humidity = cities['Humidity'].astype(float)
max_humidity = humidity.max()
locations = cities[['Lat', 'Lng']]

In [10]:
# General map creation 
fig = gmaps.figure(zoom_level=4,center=(15,25))
# adding the heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# creating the df for the ideal hotel locations
cities[['Max Temp','Wind Speed','Cloudiness']].astype(float)
fit_conditions = cities.loc[(cities['Max Temp'] < 80)] 
fit_conditions = fit_conditions.loc[(cities['Max Temp'] > 70)]
fit_conditions = fit_conditions.loc[(cities['Wind Speed'] < 10)]
fit_conditions = fit_conditions.loc[(cities['Cloudiness'] == 0)]
hotel_name = fit_conditions.dropna().set_index('City').reset_index()
# printing out the locations which fit the conditions
hotel_df

NameError: name 'hotel_df' is not defined

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# stores hotel name 
hotel_df["Hotel Name"] = " "

NameError: name 'hotel_df' is not defined

In [93]:
# setting the parameters for the google search
parameters = {
    'radius': 5000,
    'keyword': "hotel",
    'key': g_key
}
# creating the loop which will pull my information
for index,row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    
    # adding a new location with each loop pass
    parameters['location'] = f'{lat},{long}'
    
    # searching the URL for hotels
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    near_hotel = requests.get(base_url, params=parameters)
    near_hotel = near_hotel.json()
    
hotel_df


,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,ponta do sol,70,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
1,cidreira,88,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
2,morondava,138,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
3,vaini,139,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
4,nador,176,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
5,mogok,297,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
6,birjand,444,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [3]:
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")].set_index("City").reset_index()
hotel_df

NameError: name 'hotel_df' is not defined

In [94]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lng']]

In [95]:
# Add marker layer ontop of heat map
fig = gmaps.figure(map_type = 'TERRAIN')
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))